In [3]:
# %load spk_softmax_fuel.py
import htkmfc
import os
import lasagne
import time
import theano
import theano.tensor as T
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import ipdb

from fuel.datasets import IndexableDataset
from fuel.schemes import ShuffledScheme
from fuel.streams import DataStream

from collections import OrderedDict
import h5py

from fuel.converters.base import fill_hdf5_file

def load_dataset():
    
    f1 = open('/Users/gautamB/dnn/Code_/spk_softmax/Train_feats_labs.plst')
    lines = f1.readlines()
    lines = [l.strip() for l in lines]
    labelz = [int(l.split()[1]) for l in lines] 
    #labelz = labelz[:20]
    features = [l.split()[0] for l in lines] 
    
    f2 = open('/Users/gautamB/dnn/Code_/spk_softmax/Valid_feats_labs.plst')
    lines = f2.readlines()
    lines = [l.strip() for l in lines]
    val_labelz = [int(l.split()[1]) for l in lines] 
    #val_labelz = val_labelz[:20]
    val_features = [l.split()[0] for l in lines] 
    
    n_samp = len(features)
    maxlen=600 #pad all utterances to this length
    feat_dim=20
    nSpk = 98
    dpth = '/Users/gautamB/0101_data/VQ_VAD_HO_EPD/'

    Data = np.zeros((n_samp, maxlen, feat_dim), dtype='float32')
    Mask = np.zeros((n_samp,maxlen), dtype='float32')
    #Targets = np.zeros((n_samp, nSpk), dtype='int32')
    
    vn_samp = len(val_features)
    val_Data = np.zeros((vn_samp, maxlen, feat_dim), dtype='float32')
    val_Mask = np.zeros((vn_samp,maxlen), dtype='float32')
    #Targets = np.zeros((n_samp, nSpk), dtype='int32')

    for ind,f in enumerate(features):
        fname = os.path.join(dpth,f+'.fea')
        fi = htkmfc.HTKFeat_read(fname)
        data = fi.getall()[:,:20]
        Mask[ind,:data.shape[0]] = 1.0
        pad = maxlen - data.shape[0]
        data = np.vstack((data, np.zeros((pad,20), dtype='float32')))
        Data[ind,:,:] = data
        

    for ind,f in enumerate(val_features):
        fname = os.path.join(dpth,f+'.fea')
        fi = htkmfc.HTKFeat_read(fname)
        data = fi.getall()[:,:20]
        val_Mask[ind,:data.shape[0]] = 1.0
        pad = maxlen - data.shape[0]
        data = np.vstack((data, np.zeros((pad,20), dtype='float32')))
        val_Data[ind,:,:] = data


    return Data, Mask, np.asarray(labelz, dtype='int32'), val_Data, val_Mask, np.asarray(val_labelz, dtype='int32')

Data, mask, labelz, val_data, val_mask, val_labels = load_dataset()

In [11]:
f = h5py.File('dataset.hdf5', mode='w')
data = (('train', 'features', Data),
        ('train', 'mask', mask),
        ('train', 'targets', labelz),
        ('valid', 'features', val_data),
        ('valid', 'mask', val_mask),
        ('valid', 'targets', val_labels))
fill_hdf5_file(f, data)

for i, label in enumerate(('batch', 'maxlen', 'feat_dim')):
    f['features'].dims[i].label = label
for i, label in enumerate(('batch', 'maxlen')):
    f['mask'].dims[i].label = label
for i, label in enumerate(('batch',)):
    f['targets'].dims[i].label = label

In [12]:
f.flush()
f.close()

In [13]:
from fuel.datasets import H5PYDataset

# Min/max sequence length
MAX_LENGTH = 600
# Number of units in the hidden (recurrent) layer
N_HIDDEN = 100
F_DIM = 20
# Number of training sequences ain each batch
N_BATCH = 1
# Optimization learning rate
LEARNING_RATE = .001
# All gradients above this will be clipped
GRAD_CLIP = 100
# How often should we check the output?
EPOCH_SIZE = 100
# Number of epochs to train the net
NUM_EPOCHS = 10
BATCH_SIZE = 5
nSpk = 98

X = T.tensor3(name='input',dtype='float32')
Mask = T.matrix(name = 'mask', dtype='float32')

target = T.matrix(name='target_values', dtype='float32')

print("Building network ...")

l_in = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH, F_DIM), input_var = X)
n_batch,_,_ = l_in.input_var.shape
#print lasagne.layers.get_output(l_in, inputs={l_in: X}).eval({X: x_dummy}).shape

l_mask = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH), input_var = Mask)
#print lasagne.layers.get_output(l_mask, inputs={l_mask: Mask}).eval({Mask: mask}).shape

#initialize the gates
l_forward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, only_return_final=True)
l_backward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, only_return_final=True,
                                    backwards=True)
l_sum = lasagne.layers.ConcatLayer([l_forward, l_backward])

#l_proj = lasagne.layers.DenseLayer(l_concat, num_units=64, nonlinearity=lasagne.nonlinearities.linear)

l_softmax = lasagne.layers.DenseLayer(l_sum, num_units=nSpk, nonlinearity=lasagne.nonlinearities.softmax)
#print lasagne.layers.get_output(l_softmax, inputs={l_in: X, l_mask: Mask}).eval({X: d1, Mask: m1}).shape
labels = T.ivector(name='labels')

network_output = lasagne.layers.get_output(l_softmax)
val_prediction = lasagne.layers.get_output(l_softmax, deterministic=True)
#needed for accuracy
#don't use the one hot vectors here
#The one hot vectors are needed for the categorical cross-entropy
val_acc = T.mean(T.eq(T.argmax(val_prediction, axis=1), labels), dtype=theano.config.floatX)
#training accuracy
train_acc = T.mean(T.eq(T.argmax(network_output, axis=1), labels), dtype=theano.config.floatX)


#T.argmax(network_output, axis=1).eval({X: d1, Mask: m1})

#print network_output.eval({X: d1, Mask: m1})[1][97]
#cost function
total_cost = lasagne.objectives.categorical_crossentropy(network_output, labels)
#total_cost = -(labels*T.log(network_output) + (1-labels)*T.log(1-network_output)) 
mean_cost = total_cost.mean()
#accuracy function
val_cost = lasagne.objectives.categorical_crossentropy(val_prediction, labels)
val_mcost = val_cost.mean()

#Get parameters of both encoder and decoder
all_parameters = lasagne.layers.get_all_params([l_softmax], trainable=True)

print("Trainable Model Parameters")
print("-"*40)
for param in all_parameters:
    print(param, param.get_value().shape)
print("-"*40)
#add grad clipping to avoid exploding gradients
all_grads = [T.clip(g,-5,5) for g in T.grad(mean_cost, all_parameters)]
all_grads = lasagne.updates.total_norm_constraint(all_grads,5)

updates = lasagne.updates.adam(all_grads, all_parameters, learning_rate=0.005)

train_func = theano.function([X, Mask, labels], [mean_cost, train_acc], updates=updates)

val_func = theano.function([X, Mask, labels], [val_mcost, val_acc])


num_epochs=100

train_set = H5PYDataset('dataset.hdf5', which_sets=('train',))
valid_set = H5PYDataset('dataset.hdf5', which_sets=('valid',))


#load the dataset
#Data, Msk, Targets, val_Data, val_Msk, val_tars = load_dataset()

#train_set = IndexableDataset(
#    indexables = OrderedDict([('features', Data), ('mask',Msk), ('targets', Targets)]), 
#    axis_labels={'features':('batch','maxlen','feat_dim'),'mask':('batch','maxlen'), 'targets':('batch','index')})

#valid_set = IndexableDataset(
#    indexables = OrderedDict([('features', val_Data), ('mask', val_Msk), ('targets', val_tars)]), 
#    axis_labels={'features':('batch','maxlen','feat_dim'),'mask':('batch','maxlen'), 'targets':('batch','index')})


trainerr=[]

print("Starting training...")
    # We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    tr_acc = 0
    train_batches = 0
    
    t_state = train_set.open()
    v_state = valid_set.open()

    scheme = ShuffledScheme(examples=train_set.num_examples, batch_size=256)
    scheme1 = ShuffledScheme(examples=valid_set.num_examples, batch_size=128)


    train_stream = DataStream(dataset=train_set, iteration_scheme=scheme)
    valid_stream = DataStream(dataset=valid_set, iteration_scheme=scheme1)
    
    start_time = time.time()
    
    for data in train_stream.get_epoch_iterator():
        t_data, t_mask, t_labs = data
        terr, tacc = train_func(t_data, t_mask, t_labs)
        train_err += terr
        tr_acc += tacc
        train_batches += 1
        
    val_err = 0
    val_acc = 0
    val_batches = 0
    
    for data in valid_stream.get_epoch_iterator():
        v_data, v_mask, v_tars = data
        err, acc = val_func(v_data, v_mask ,v_tars)
        val_err += err
        val_acc += acc
        val_batches += 1
        
    trainerr.append(train_err/train_batches)
    
    train_set.close(state=t_state)
    valid_set.close(state=v_state)
        
# Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
    epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  training accuracy:\t\t{:.2f} %".format(
        tr_acc / train_batches * 100))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
import cPickle
#Save the final model
spth = '/misc/data15/reco/bhattgau/Rnn/Projects/Rvector/Weights/basic-softmax'

tfile = open('/misc/data15/reco/bhattgau/Rnn/Projects/Rvector/Weights/basic-softmax/trainerr.npy','wb')
trainerr = np.asarray(trainerr, dtype='float32')
np.save(tfile,trainerr)

print('Saving Model ...')
model_params = lasagne.layers.get_all_param_values(l_softmax)
model_name = 'basic_softmax_500' + '.pkl'
vpth = os.path.join(spth, model_name)
fsave = open(vpth,'wb')  
cPickle.dump(model_params, fsave, protocol=cPickle.HIGHEST_PROTOCOL)
fsave.close()

Building network ...
Trainable Model Parameters
----------------------------------------
(W_in_to_updategate, (20, 100))
(W_hid_to_updategate, (100, 100))
(b_updategate, (100,))
(W_in_to_resetgate, (20, 100))
(W_hid_to_resetgate, (100, 100))
(b_resetgate, (100,))
(W_in_to_hidden_update, (20, 100))
(W_hid_to_hidden_update, (100, 100))
(b_hidden_update, (100,))
(W_in_to_updategate, (20, 100))
(W_hid_to_updategate, (100, 100))
(b_updategate, (100,))
(W_in_to_resetgate, (20, 100))
(W_hid_to_resetgate, (100, 100))
(b_resetgate, (100,))
(W_in_to_hidden_update, (20, 100))
(W_hid_to_hidden_update, (100, 100))
(b_hidden_update, (100,))
(W, (200, 98))
(b, (98,))
----------------------------------------
Starting training...
Epoch 1 of 100 took 89.887s
  training loss:		4.382030
  training accuracy:		7.13 %
  validation loss:		4.198010
  validation accuracy:		8.68 %
Epoch 2 of 100 took 79.312s
  training loss:		3.316444
  training accuracy:		34.07 %
  validation loss:		3.723460
  validation accura

KeyboardInterrupt: 

In [8]:
f1 = open('/misc/data15/reco/bhattgau/Rnn/Lists/all_audio.plst')
lines = f1.readlines()
lines = [l.strip() for l in lines]
all_ids = [l.split('_')[0] for l in lines]
spk_ids = list(set(all_ids))

labs = []

for id in all_ids:
    labs.append(spk_ids.index(id))

In [58]:
import random
import itertools
#f2 = open('/misc/data15/reco/bhattgau/Rnn/Lists/spk_softmax/All_feats_labs.plst','wb')
mix = zip(lines, labs)
VALID=[]
ALL=[]

for id in spk_ids:
    tmp=[]
    for l in lines:
        if l.split('_')[0] == id:
            tmp.append(l)
    ALL.append(tmp)

In [60]:
for a in ALL[:50]:
    val = itertools.islice(a,4)
    val = list(val)
    VALID.append(val)

In [64]:
#This is the list of validation files
V2=[]
for v in VALID:
    for vv in v:
        V2.append(vv)

In [68]:
ALL2 = []
for a in ALL:
    for aa in a:
        ALL2.append(aa)

In [71]:
#This is the list of training files
TRAIN=[]

for a in ALL2:
    if a not in V2:
        TRAIN.append(a)
        

In [74]:
#now we need to give them labels
train_labs=[]
for t in TRAIN:
    tlab = t.split('_')[0]
    train_labs.append(spk_ids.index(tlab))

In [77]:
valid_labs = []
for v in V2:
    vlab = v.split('_')[0]
    valid_labs.append(spk_ids.index(vlab))

In [85]:
zz = zip(V2, valid_labs)
random.shuffle(zz)


In [86]:
#randomize and write these to files
f2 = open('/misc/data15/reco/bhattgau/Rnn/Lists/spk_softmax/Valid_feats_labs.plst','wb')
#t_mix = 
for T,TL in zz:
    f2.write('%s %s\n'%(T,TL))
f2.close()

In [29]:
        
        #now select 3 from this list for the validation list
        valid = itertools.islice(tmp,4)
        valid = list(valid)
        VALID.append(valid)

In [25]:
import itertools

top3 = itertools.islice(tmp,3)
top3 = list(top3)

In [78]:
len(train_labs)

1327

In [2]:
# %load ../mnist.py
#!/usr/bin/env python

"""
Usage example employing Lasagne for digit recognition using the MNIST dataset.

This example is deliberately structured as a long flat file, focusing on how
to use Lasagne, instead of focusing on writing maximally modular and reusable
code. It is used as the foundation for the introductory Lasagne tutorial:
http://lasagne.readthedocs.org/en/latest/user/tutorial.html

More in-depth examples and reproductions of paper results are maintained in
a separate repository: https://github.com/Lasagne/Recipes
"""

from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne


# ################## Download and prepare the MNIST dataset ##################
# This is just some way of getting the MNIST dataset from an online location
# and loading it into numpy arrays. It doesn't involve Lasagne at all.

def load_dataset():
    # We first define some helper functions for supporting both Python 2 and 3.
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
        import cPickle as pickle

        def pickle_load(f, encoding):
            return pickle.load(f)
    else:
        from urllib.request import urlretrieve
        import pickle

        def pickle_load(f, encoding):
            return pickle.load(f, encoding=encoding)

    # We'll now download the MNIST dataset if it is not yet available.
    url = 'http://deeplearning.net/data/mnist/mnist.pkl.gz'
    filename = 'mnist.pkl.gz'
    if not os.path.exists(filename):
        print("Downloading MNIST dataset...")
        urlretrieve(url, filename)

    # We'll then load and unpickle the file.
    import gzip
    with gzip.open(filename, 'rb') as f:
        data = pickle_load(f, encoding='latin-1')

    # The MNIST dataset we have here consists of six numpy arrays:
    # Inputs and targets for the training set, validation set and test set.
    X_train, y_train = data[0]
    X_val, y_val = data[1]
    X_test, y_test = data[2]

    # The inputs come as vectors, we reshape them to monochrome 2D images,
    # according to the shape convention: (examples, channels, rows, columns)
    X_train = X_train.reshape((-1, 1, 28, 28))
    X_val = X_val.reshape((-1, 1, 28, 28))
    X_test = X_test.reshape((-1, 1, 28, 28))

    # The targets are int64, we cast them to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_val = y_val.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_val, y_val, X_test, y_test

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

In [6]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [ ]:
# ##################### Build the neural network model #######################
# This script supports three types of models. For each one, we define a
# function that takes a Theano variable representing the input and returns
# the output layer of a neural network model build in Lasagne.

def build_mlp(input_var=None):
    # This creates an MLP of two hidden layers of 800 units each, followed by
    # a softmax output layer of 10 units. It applies 20% dropout to the input
    # data and 50% dropout to the hidden layers.

    # Input layer, specifying the expected input shape of the network
    # (unspecified batchsize, 1 channel, 28 rows and 28 columns) and
    # linking it to the given Theano variable `input_var`, if any:
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                     input_var=input_var)

    # Apply 20% dropout to the input data:
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)

    # Add a fully-connected layer of 800 units, using the linear rectifier, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    l_hid1 = lasagne.layers.DenseLayer(
            l_in_drop, num_units=800,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # We'll now add dropout of 50%:
    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    # Another 800-unit layer:
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1_drop, num_units=800,
            nonlinearity=lasagne.nonlinearities.rectify)

    # 50% dropout again:
    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)

    # Finally, we'll add the fully-connected output layer, of 10 softmax units:
    l_out = lasagne.layers.DenseLayer(
            l_hid2_drop, num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)

    # Each layer is linked to its incoming layer(s), so we only need to pass
    # the output layer to give access to a network in Lasagne:
    return l_out


def build_custom_mlp(input_var=None, depth=2, width=800, drop_input=.2,
                     drop_hidden=.5):
    # By default, this creates the same network as `build_mlp`, but it can be
    # customized with respect to the number and size of hidden layers. This
    # mostly showcases how creating a network in Python code can be a lot more
    # flexible than a configuration file. Note that to make the code easier,
    # all the layers are just called `network` -- there is no need to give them
    # different names if all we return is the last one we created anyway; we
    # just used different names above for clarity.

    # Input layer and dropout (with shortcut `dropout` for `DropoutLayer`):
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)
    if drop_input:
        network = lasagne.layers.dropout(network, p=drop_input)
    # Hidden layers and dropout:
    nonlin = lasagne.nonlinearities.rectify
    for _ in range(depth):
        network = lasagne.layers.DenseLayer(
                network, width, nonlinearity=nonlin)
        if drop_hidden:
            network = lasagne.layers.dropout(network, p=drop_hidden)
    # Output layer:
    softmax = lasagne.nonlinearities.softmax
    network = lasagne.layers.DenseLayer(network, 10, nonlinearity=softmax)
    return network


def build_cnn(input_var=None):
    # As a third model, we'll create a CNN of two convolution + pooling stages
    # and a fully-connected hidden layer in front of the output layer.

    # Input layer, as usual:
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)
    # This time we do not apply input dropout, as it tends to work less well
    # for convolutional layers.

    # Convolutional layer with 32 kernels of size 5x5. Strided and padded
    # convolutions are supported as well; see the docstring.
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
    # Expert note: Lasagne provides alternative convolutional layers that
    # override Theano's choice of which implementation to use; for details
    # please see http://lasagne.readthedocs.org/en/latest/user/tutorial.html.

    # Max-pooling layer of factor 2 in both dimensions:
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # A fully-connected layer of 256 units with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify)

    # And, finally, the 10-unit output layer with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)

    return network


# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


# ############################## Main program ################################
# Everything else will be handled in our main program now. We could pull out
# more functions to better separate the code, but it wouldn't make it any
# easier to read.

def main(model='mlp', num_epochs=500):
    # Load the dataset
    print("Loading data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

    # Prepare Theano variables for inputs and targets
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    # Create neural network model (depending on first command line parameter)
    print("Building model and compiling functions...")
    if model == 'mlp':
        network = build_mlp(input_var)
    elif model.startswith('custom_mlp:'):
        depth, width, drop_in, drop_hid = model.split(':', 1)[1].split(',')
        network = build_custom_mlp(input_var, int(depth), int(width),
                                   float(drop_in), float(drop_hid))
    elif model == 'cnn':
        network = build_cnn(input_var)
    else:
        print("Unrecognized model type %r." % model)

    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()
    # We could add some weight decay as well here, see lasagne.regularization.

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.01, momentum=0.9)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    # As a bonus, also create an expression for the classification accuracy:
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    # Finally, launch the training loop.
    print("Starting training...")
    # We iterate over epochs:
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

    # After training, we compute and print the test error:
    test_err = 0
    test_acc = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        test_err += err
        test_acc += acc
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
    print("  test accuracy:\t\t{:.2f} %".format(
        test_acc / test_batches * 100))

    # Optionally, you could now dump the network weights to a file like this:
    # np.savez('model.npz', lasagne.layers.get_all_param_values(network))


if __name__ == '__main__':
    if ('--help' in sys.argv) or ('-h' in sys.argv):
        print("Trains a neural network on MNIST using Lasagne.")
        print("Usage: %s [MODEL [EPOCHS]]" % sys.argv[0])
        print()
        print("MODEL: 'mlp' for a simple Multi-Layer Perceptron (MLP),")
        print("       'custom_mlp:DEPTH,WIDTH,DROP_IN,DROP_HID' for an MLP")
        print("       with DEPTH hidden layers of WIDTH units, DROP_IN")
        print("       input dropout and DROP_HID hidden dropout,")
        print("       'cnn' for a simple Convolutional Neural Network (CNN).")
        print("EPOCHS: number of training epochs to perform (default: 500)")
    else:
        kwargs = {}
        if len(sys.argv) > 1:
            kwargs['model'] = sys.argv[1]
        if len(sys.argv) > 2:
            kwargs['num_epochs'] = int(sys.argv[2])
        main(**kwargs)

In [1]:
import os
import numpy as np
import htkmfc
import random

def data_dict(audio):

    f=open(audio)
    dpth = '/misc/data15/reco/bhattgau/Rnn/Data/mfcc/Nobackup/VQ_VAD_HO_EPD/'

    lines=f.readlines()
    Data = {}
    maxlen=600 #pad all utterances to this length
    feat_dim=20

    for l in lines:
        l=l.strip()
        fname = os.path.join(dpth,l+'.fea')
        fi = htkmfc.HTKFeat_read(fname)
        data = fi.getall()[:,:20]
        pad = np.zeros(((maxlen-data.shape[0]), 20), dtype='float32')
        data = np.vstack((data,pad))
        Data[l] = data
        
    return Data

In [28]:
au= '/misc/data15/reco/bhattgau/Rnn/Lists/Validspks.plst'

In [29]:
f = open(au)
utts = f.readlines()
utts = [u.strip() for u in utts]
all_ids = [t.split('_')[0] for t in utts]
spk_ids = list(set([t.split('_')[0] for t in utts]))

In [30]:
labels=[]
for ids in all_ids:
    labels.append(spk_ids.index(ids))

In [31]:
import random
f1 = open('/misc/data15/reco/bhattgau/Rnn/Lists/Validspks_labs.plst','wb')
snl = zip(utts,labels)
random.shuffle(snl)

for utt,lab in snl:
    f1.write('%s %s\n'%(utt,lab))
f1.close()

In [4]:
#we have 98 speaker models
labels=[]
for idx in spk_ids:
    labels.append(spk_ids.index(idx))

In [14]:
import random
g = random.sample(r,3)

NameError: name 'r' is not defined

In [5]:
#this will be a generator()
slots=[]
maxlen=600
feat_dim=20
nSpk = 98
labels=[]
ptr=0

X_data1 = np.zeros((nSpk,maxlen,feat_dim), dtype='float32')
X_data2 = np.zeros((nSpk,maxlen,feat_dim), dtype='float32')
X_data3 = np.zeros((nSpk,maxlen,feat_dim), dtype='float32')

for lab,idx in enumerate(spk_ids):
    labels.append(lab)    
    for k,i in enumerate(data.keys()):
        spk = i.split('_')[0]
        if spk == idx:
            slots.append(k)
            
        
    slot = np.asarray(slots)          
    l = random.sample(slot,3)

    enr1 = data.keys()[l[0]]
    enr2 = data.keys()[l[1]]
    enr3 = data.keys()[l[2]]

    en_data1 = data[enr1]
    en_data2 = data[enr2]
    en_data3 = data[enr3]
    
    X_data1[ptr,:,:] = en_data1
    X_data2[ptr,:,:] = en_data2
    X_data3[ptr,:,:] = en_data3
      
    ptr+=1

In [7]:
#generate the corresponding masks
mask1 = np.zeros((nSpk,maxlen), dtype='float32')
mask2 = np.zeros((nSpk,maxlen), dtype='float32')
mask3 = np.zeros((nSpk,maxlen), dtype='float32')

for i in range(nSpk):
    
    ones1 = X_data1[i][:,1][np.nonzero(X_data1[i][:,1])]
    ones2 = X_data2[i][:,1][np.nonzero(X_data2[i][:,1])]
    ones3 = X_data3[i][:,1][np.nonzero(X_data3[i][:,1])]
    
    mask1[i,:ones1.shape[0]] = 1.0
    mask2[i,:ones2.shape[0]] = 1.0
    mask3[i,:ones3.shape[0]] = 1.0

In [19]:
s1 = zip(X_data1, mask1,labels, X_data2, mask2,labels, X_data3, mask3,labels)
random.shuffle(s1)

In [25]:
s1[2][8]

28

In [9]:
DATA = np.vstack((X_data1, X_data2, X_data3))
MASK = np.vstack((mask1, mask2, mask3))

In [25]:
x_data = np.asarray(x_data)

In [26]:
x_data.shape

(98, 600, 20)

In [8]:
import theano
import theano.tensor as T

Using gpu device 0: GeForce GTX 680 (CNMeM is disabled)


In [10]:
data1 = theano.shared(np.asarray(DATA, dtype=theano.config.floatX), borrow=True)

In [11]:
d1 = data1[:98,:,:]
d2 = data1[98:196,:,:]
d3 = data1[196:,:,:]

In [13]:
for i in range(98):
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


Each of the data matrices will be loaded as theano shared variables (data, masks and labels)
Shuffle and load them, and then slice mini-batches out of each
e.g. 16,16,16 so a batch size of 48
After forward propogating throught the RNN, we need to do some grouping, before decision making

In [36]:
import os
import numpy as np
import htkmfc
import random

dpth = '/misc/data15/reco/bhattgau/Rnn/Data/mfcc/Nobackup/VQ_VAD_HO_EPD/'

feat_dim = 20
maxlen=600 #pad all utterances to this length

f1 = open('/misc/data15/reco/bhattgau/Rnn/Lists/Allspks_labs.plst')
lines = f1.readlines()
lines = [l.strip() for l in lines]

audio = [l.split()[0] for l in lines]
labs = [l.split()[1] for l in lines]

In [39]:
Data = np.zeros((len(audio), maxlen, feat_dim), dtype='float32')

for ind,l in enumerate(audio):
    fname = os.path.join(dpth,l+'.fea')
    fi = htkmfc.HTKFeat_read(fname)
    data = fi.getall()[:,:20]
    pad = np.zeros(((maxlen-data.shape[0]), 20), dtype='float32')
    data = np.vstack((data,pad))
    Data[ind,:,:] = data

In [41]:
train_set = theano.shared(Data, borrow=True)

In [43]:
train_labs = theano.shared(np.asarray(labs, dtype=theano.config.floatX), borrow=True)